# easyPCA: PCA and cluster analysis
Principal component analysis (PCA) with optional classification and HDBSCAN clustering in the PCA score plot.

Jupyter Notebook is provided in https://github.com/danield5732/easyPCA

### todos

very nice to have, but still nice to have:
- plot PCA Score graph with ploty to enable hoover option
- include loadings graph for selected PCs to Score plot block

In [ ]:
# from Lucas: Dat1_pretreatment_v7 and Michael Galarnyk: PCA_Data_Visualization_Iris_Dataset_Blog
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm
import matplotlib.colors
import seaborn as sns
import scipy.signal
import os
import sklearn.preprocessing
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
import ipywidgets as widgets
import plotly.graph_objs as go
import plotly as py
from mpl_toolkits.mplot3d import Axes3D 
#import umap   # https://umap-learn.readthedocs.io/en/latest/
from matplotlib.patches import Rectangle

### Load DataFrame from Excel

In [ ]:
# load df from Excel
# dtype is not interpreted and therefore, preserve accuracy
df = pd.read_excel('PCA-FSA_03.xlsx', index_col = 0, header = 0, dtype = 'object')   
df

### Define feature and classifier columns

In [ ]:
features = ['HK 950', 'HKP 1050', 'CCP 90D', '5000 P-f', 'AZ 1050', 'PS WP 235', 'CC 401']
classifiers = ['analytics', 'category']

#features = df.drop(classifiers, axis = 1).columns
#classifiers = df.drop(features, axis = 1).columns

features

## Preselect sample sets from loaded DataFrame

#### filter for certain criteria or analyze the whole DataFrame

In [ ]:
#df_select = df.copy()

#mask = ((df.loc[:,'source'].values == 'Pia_PROTECT') & 
#        (df.loc[:,'washed'].values != 'washed'))

#mask = (df.loc[:,'washed'].values != 'washed')

df_select = df.loc[mask,(list(features) + list(classifiers))].copy()

#### Handle non numeric values in feature columns

In [ ]:
# set strings in feature columns to NaN
for feature in features:
    df_select.loc[:,feature] = pd.to_numeric(df_select.loc[:,feature], errors='coerce')

#### Handle NaN values in classifier columns

In [ ]:
# set NaN values in classifier columns to unassigned
df_select.loc[:,classifiers] = df_select.loc[:,classifiers].fillna('unassigned')

#### Handle NaN values in feature columns (Think about standardization and normalization in advance, see below!)

In [ ]:
# exclude samples with NaN in feature values
df_select = df_select.dropna(subset = features)

### Show selected data

In [ ]:
df_select

## Standardization and normaliazion of the data: What are you looking for?

There are several ways for data pretreatment. 
This section will lead you through the common options (at least for me).

To achieve what you are actual looking for, it is necessary to think about it in advance.
Otherwise, you will get a result anyway, but it is even harder to interprete.

In [ ]:
# extract feature data from selected DataFrame
x = df_select.loc[:, features].values

### Features and Samples
In general, features are located in columns and samples in rows.
If you want to swap this, you have to transpose your input data first.

#### When is it usefull to treat your samples as features?
- When you are interested in your samples, you should leave them as rows and you will see samples with similar features clustered in the scores plot.
- When you want to switch the perspective to see how your chosen features are distributed through your samples, you could think about the previous transponation.


- This directly effects the number of dots you will find in the PCA Scores plot, that originate from the number of rows.


### Standardize or normalize column-wise (features)
Performing a standardization or normalization along your features, will differentiate your samples along each feature.
Meaning the sample with the lowest feature value will stay the smales and the biggest will stay the biggest, but they are scaled into a range of e.g. -1 to 1.
Since this is done for every feature, the features will contribute to the PCA result equally.
Otherwise, features with larger scales would be overrepresented and may suppress the impact of other features.

**This is usefill,**
- if your features are independent of each other or having different units.
- if you want to see the effects of minor properties represented by a feature.

**This may not be usefull,**
- if your features are already similar and their values are directly comparable to each other.
- if your features with small values have high uncertainties that will be scaled up, too and therefore, impair the PCA result.
- if your feature values are related to sample properties that are nor represented by any feature, e.g. sample weight. Since, then the values are not comparable directly.
- if you have NaN values in your features. These rows need to be exluded or to be set to a fixed value. The latter would effect the PCA result by an unjustified manner, for my point of view.

#### Standardization vs. normalization
**Standardization** only can be done column-wise, since it centers the data by shifting the mean value to 0.
Further, it squeezes the data by setting the standard deviation to -1 and 1.
Therefore, your data is assumed to be distributed normaly according to Gauss and the distance between each value is no longer relative the same as in the original data.
If the assumption is true, your data point in the PCA results may be better clustered than by normalization.

On the other hand, **normalization** keeps the relative distances between your data by only scaling them to certain range. To center the data is recommended anyway.
There are different approaches for normalization:
- norm = 'max': the lowest value is set to 0 and the highest value is set to 1, therefore the maximum variance within the data is obtained. On the contraty, information about the minimum value is lost, if it was already 0 or only the smalles value of the feature.
- *There should be an max-norm respecting 0 as 0 and only scale by the maximum value.*
- norm = 'l1': I have no idea what is going on.
- norm = 'l2': I have no idea what is going on.


In [ ]:
# normalize and center the data. Options for norm are 'l1', 'l2' or 'max'
# 'max' is stretching the values beeing the smallest 0
x = normalize(x, norm='max', axis=0)

### Normalize row-wise (samples)
Normalization of a sample's features will create a sample specific pattern.
Standardization is not available row-wise.

**This is usefill,**
- if your features are comprised of absolute values not respecting specific differences within the samples, e.g. sample weight. This will rule out diffences in the samples that may occure independent of its features and compare the samples specific feature pattern.

**This may not be usefull,**
- if you want to see differences within on feature in several samples, since this relation gets totally lost.

Centering of the data is done anyway by the method used in the PCA class from the decomposition module in the sklearn (scikit-learn) package.

### Show data ready to be used in PCA analysis

In [ ]:
pd.DataFrame(data = x, columns = features)

## PCA and clustering of the selected and pretreated data

Since PCA yields a feature subspace that maximizes the variance along the axes, it makes sense to standardize the data, especially, if it was measured on different scales.

In [ ]:
def PC_labels(pca_explvar):
    names = ['PC']*len(pca_explvar)
    for counter, value in enumerate(names):
        names[counter] = value+str(counter+1)
    return names

#t = spec_sg_norm[spec_sg_norm['thresh_0.06']==True].iloc[:,10:len(xaxis)-10]
#t = np.array(t, dtype=np.float32)
t = x

if(min(len(features), len(df_select))>9):
    n_components = 9
else:
    n_components = min(len(features), len(df_select)) -1

pca          = PCA(n_components = n_components)
pca_scores   = pca.fit_transform(t)
pca_loadings = pca.components_
pca_explvar  = pca.explained_variance_ratio_

pca_scores   = pd.DataFrame(pca_scores)
pca_scores.columns  = PC_labels(pca_explvar)
legend        = PC_labels(pca_explvar)

In [ ]:
import hdbscan
clusterer = hdbscan.HDBSCAN(algorithm='best', alpha=1.0, approx_min_span_tree=True,
                            gen_min_span_tree=False, leaf_size=40,
                            metric='euclidean', min_cluster_size=3, min_samples=2, p=None)

In [ ]:
#legend = PC_labels(pca_explvar)
#d      = {}
#for index, value in enumerate(pca_scores.columns.values[:-1]):
#    d[value] = index
    
# the widgets
PC_x = widgets.Dropdown(options=pca_scores.columns, value=pca_scores.columns[0], description='PC_x: ', disabled=False,)
PC_y = widgets.Dropdown(options=pca_scores.columns, value=pca_scores.columns[1], description='PC_y: ', disabled=False,)
PC_class = widgets.Dropdown(options = ['None'] + list(classifiers) + ['Cluster: HDBSCAN'], value='None', description='classify: ', disabled=False,)


def update_scores(PC_x, PC_y, PC_class):
    #PCx = d[PC_x]
    #PCy = d[PC_y]
    
    fig, ax = plt.subplots(ncols=1, nrows=1,figsize=(8,8))
    
    if (PC_class == 'Cluster: HDBSCAN'):
        s = np.array(pca_scores[[PC_x, PC_y]])   # get current scores
        hdb = clusterer.fit(s)                   # perform clustering
        hdbscan = clusterer.labels_              # get cluster results
        hdbscan = hdbscan+1                      # shift values, beeing unassignes -1: 0 
        hdbscan = ['Cluster ' + s for s in hdbscan.astype(str)]   # set values to strings and modify
        hdbscan = np.array(hdbscan)              # retransform list to np.array
        hdbscan = np.where(hdbscan == 'Cluster 0', 'unassigned', hdbscan)   # rename Cluster 0 to 'unassigned'
        classification = hdbscan
        df_select.loc[:,'HDBSCAN: '+PC_x+'-'+PC_y] = hdbscan
    elif (PC_class == 'None'):
        classification = ['PCA Scores']
    else:
        classification = df_select[PC_class].values
    
    targets = sorted(list(set(classification)))
    for target in targets:
        if (PC_class == 'None'):
            indicesToKeep = pca_scores.index
        else:
            indicesToKeep = classification == target
                
        ax.scatter(pca_scores.loc[indicesToKeep,PC_x],
                    pca_scores.loc[indicesToKeep,PC_y],
                    s = 75)

    ax = fig.gca()
    #ax.grid()
    ax.hlines(y=0, xmin = min(pca_scores.loc[:,PC_x]), xmax = max(pca_scores.loc[:,PC_x]), color='g', linestyle='dotted')
    ax.vlines(x=0, ymin = min(pca_scores.loc[:,PC_y]), ymax = max(pca_scores.loc[:,PC_y]), color='g', linestyle='dotted')
    ax.set_xlabel(PC_x, fontsize = 15)
    ax.set_ylabel(PC_y, fontsize = 15)
    ax.set_title('PCA Score plot', fontsize = 20)
    
    ax.legend(targets)
    
    plt.tight_layout()

In [ ]:
legend = PC_labels(pca_explvar)
# the widgets
loadings = widgets.SelectMultiple(options = np.array(range(len(pca_explvar)), dtype=int), value = (0, ),
                                  description = 'Loadings: ')

def update_loadings(loadings):
    fig = plt.figure(figsize=(10,8))
    ax2 = plt.subplot2grid((2, 1), (0,0), colspan=2)
    ax3 = plt.subplot2grid((2, 1), (1,0), colspan=2)

    # plot the loadings
    leg = [legend[i] for i in loadings]
    ax2.plot(features,pca_loadings[loadings,:].T)
    #ax2.invert_xaxis()
    ax2.hlines(y=0, xmin = 0, xmax = len(pca_explvar), color='g', linestyle='dotted')
    #ax2.legend(leg, loc='center left', bbox_to_anchor=(1, 0.5) )
    ax2.set_ylabel('')
    ax2.set_title('Loadings of ' + str(leg))
    
    # plot the explained variance
    sns.lineplot(x=list(range(1,len(pca_explvar)+1)), y=pca_explvar, ax=ax3)
    sns.lineplot(x=list(range(1,len(pca_explvar)+1)), y=np.cumsum(pca_explvar), ax=ax3)
    ax3.legend(['PC_explvar','sum curve'])#, loc='center left', bbox_to_anchor=(1, 0.5) )
    ax3.set_xlabel('No. principal component')
    ax3.set_ylabel('Explained variance')
    ax3.set_title('Explained variance')
    ax3.set_xticks(np.linspace(1,len(pca_explvar),len(pca_explvar), dtype=np.int))
    ax3.set_xticklabels(np.linspace(1,len(pca_explvar),len(pca_explvar), dtype=np.int))
    ax3.set_ylim = [0,1]
    ax3.yaxis.grid(True)
    plt.tight_layout()

In [ ]:
widgets.interactive(update_scores, PC_x=PC_x, PC_y=PC_y, PC_class=PC_class)

In [ ]:
widgets.interactive(update_loadings, loadings=loadings)

In [ ]:
# data used for PCA
df_select

In [ ]:
# data of the principal components
pca_scores_save = pca_scores.copy()
pca_scores_save.set_index(df_select.index, inplace = True)
pca_scores_save = pd.concat([pca_scores_save, df_select.loc[:,list(classifiers)]], axis=1)
pca_scores_save

In [ ]:
# save PCA results to Excel
with pd.ExcelWriter('PCA_results.xlsx') as writer:
    df_select.to_excel(writer, sheet_name = 'df_select')
    pca_scores_save.to_excel(writer, sheet_name = 'pca_scores')